# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico III : Introducción al aprendizaje automático
## Dataset Creation

## Warning

> **Running this notebook to build the dataset takes apprx 2 hours.**

> The final result is saved in the file `final_big.csv`

### Dependencies

In [1]:
%%capture
!pip3 install spotipy
!pip3 install pandas
!pip3 install spacy
!pip3 install pymusixmatch
!pip3 install nltk
!pip3 install seaborn
!pip3 install requests
!pip3 install tqdm
!pip3 install plotly
!pip3 install sklearn
!pip3 install sentiment_analysis_spanish

### Imports and api access

In [2]:
import pandas as pd
import numpy as np
import spotipy
import spacy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
import tqdm
import plotly.express as px
import plotly.graph_objects as go
import requests
from collections import Counter
from nltk import ngrams, bigrams
import nltk
import itertools
import matplotlib.pyplot as plt
import json
import spacy
from sentiment_analysis_spanish import sentiment_analysis
from  statistics import median,mean


client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# es = es_core_web_sm
#python -m spacy link es_core_news_sm es
nlp = spacy.load("es")

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sentiment = sentiment_analysis.SentimentAnalysisSpanish()


In [3]:
sns.set_context(context='paper')

# Dataset Creation

This dataset is based on a collaboratory spotify playlist created by the students of the mentorship.

* It contains 1755 songs at the time of us writting this.

* All songs correspond to spanish-speaking artists. Since the playlist creators are latinamericans, most artists of the playlist are also latinamerican. This is relevant since there will be a NLP task involved in the dataset creation. See the section of Problems found to read more discussions about this point.

We need a dataset with both **music features** for each song, and **lyrics sentiment analysis**. In order to create the music features part we use Spotify's API. For the lyrics sentiment analysis we must use first a lyrics API to obtain them, and then we need to explore ways of doing sentiment analysis on them.

## Music Features

Design decisions:
    - For songs with multiple artists, we save a list of the names of all the artists.
    - The genres provided by Spotify are associated to the artist. Also, Spotify provides a *list* of genres. We save a list of all genres returned by Spotify.

In [3]:
#Aux funcs
def genres_by_artist_id(id): #id: str
    artist = sp.artist(id)
    genres = artist['genres']
    return genres    #genres: List[str]

def songs_from_album_id(album_id):
    songs = []
    album = sp.album(album_id)
    artist = album['artists'][0]['name']
    for item in album['tracks']['items']:
        track = {}
        track["song_name"] = item['name']
        track["song_id"] = item['id']
        track["album_name"] = album['name']
        track["album_id"] = album["id"]
        audio_features = sp.audio_features(track["song_id"])
        track["audio_features"] = audio_features[0]
        track["artist"] = artist
        songs.append(track)
    return songs    #songs:List[dict]

def get_genres(artists_id):
    res = [genres_by_artist_id(x) for x in artists_id if genres_by_artist_id(x)!= []]
    if res != []:
        return res[0]
    return res

def add_track(track_id, songs): #track_id:str, songs:List[dict]
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)
    row = {}
    row["song_name"] = track['name']
    row["song_id"] = track['id']
    row["artists"] = [x["name"] for x in track["artists"]]
    row["artists_id"] = [x["id"] for x in track["artists"]]
    row["album_name"] = track['album']['name']
    row["album_id"] = track['album']['id']
    row["audio_features"] = audio_features[0]
    row["genres"] = get_genres(row["artists_id"])
    songs.append(row)
    return songs


In [203]:
PLAYLIST_ID = "2IuD0qZb14cji5y52crdsO"
TEST_PLAYLIST_ID = "3gLmPh92AyeYDKYLaNC8uv"
songs = []

def add_songs_of_playlist(playlist_id,songs_array):

    print("This may take a while...")
    offset = 0
    playlist = sp.playlist_tracks(playlist_id,offset=offset,limit=100)
    batches = playlist["total"] // 100 + 1
    print("...downloading "+ str(playlist["total"]) + " songs")
    print("in "+str(batches)+ " batches")
    for j in tqdm.tqdm(range(batches)):
        for i in range(len(playlist["items"])):
            add_track(playlist["items"][i]["track"]["id"],songs_array)
        offset += len(playlist["items"])
        playlist = sp.playlist_tracks(playlist_id, offset=offset,limit=100)
    return

add_songs_of_playlist(PLAYLIST_ID,songs)

This may take a while...


  0%|          | 0/18 [00:00<?, ?it/s]

...downloading 1755 songs
in 18 batches


100%|██████████| 18/18 [31:41<00:00, 105.62s/it]


In [204]:
songs_original_df = pd.DataFrame(songs)
songs_original_df.count()

song_name         1755
song_id           1755
artists           1755
artists_id        1755
album_name        1755
album_id          1755
audio_features    1755
genres            1755
dtype: int64

In [205]:
#to save this step
#songs_original_df.to_csv("./base.csv",sep="#",index=False)


In [5]:
#songs_original_df = pd.read_csv("../base.csv",sep="#")


Checking dataframe's consistency.

First let's look for null in the genres field.

In [206]:
songs_original_df[[x == [] for x in songs_original_df["genres"]]]

,song_name,song_id,artists,artists_id,album_name,album_id,audio_features,genres
48,Cambalache,3PI0FE7JUmEmEyN5YgKPZA,[Enrique Santos Discépolo],[0aPYs7yoiP2NtS5xNZXKjg],"Enrique Santos Discepolo ""El poeta del tango"" ...",59tn7tvd1M5XNWwV3TaVWC,"{'danceability': 0.492, 'energy': 0.541, 'key'...",[]
478,Sigue Feliz,1s0ndZpf2KeKEA08CsIFia,[Alonso y Bernardo],[5sskVxLnToHrnwTAICyVF5],"Narcos, Vol. 2 (More Music from the Netflix Or...",0EJRlYjvVcym9K4wrww9vB,"{'danceability': 0.672, 'energy': 0.625, 'key'...",[]
526,Celia,1rzFbkSvxQv6r3PSGjn7Ub,[Incas de Oro],[58wFXtpJxfvtigDaRWTNcj],Los Mejores Tinkus,4xDelBtEq3aJCU8hU6gFLB,"{'danceability': 0.675, 'energy': 0.477, 'key'...",[]
600,Arteria Ulnar,7kGsNBECFyCQ0fBJn2KB6o,[Té de Brujas],[39BvzssARgDTZ1Kf0uqNfj],Arteria Ulnar,5g5rgxGPlCnRTrYyf173fp,"{'danceability': 0.497, 'energy': 0.651, 'key'...",[]
604,No Le Ganamos a Nadie,0F0I189uNvQBdgy1SFNOec,[Literal],[0Ec1MqHP5MENR7rK3DtO3G],No Le Ganamos a Nadie,43moEeCjsTjk6N25XRin0S,"{'danceability': 0.568, 'energy': 0.938, 'key'...",[]
606,Contratiempos,50GbEo3clyzJRzuAjIFWdz,[Parientes],[76lUSSvc6Z83CLrIVB7YrE],Contratiempos,3F6da9yP7HMGwl88egAqZ5,"{'danceability': 0.549, 'energy': 0.778, 'key'...",[]
613,Si Tú No Estas (Nashville),3TCpMjVi4DVzbc5dXLpEeX,[Stokoff],[03wfTeoZex93T5TPxWo3B9],Si Tú No Estas (Nashville),2fNd57gzWCMwsNVG0K5YQy,"{'danceability': 0.545, 'energy': 0.652, 'key'...",[]
617,Una Nueva Realidad,3CQinOLvOg1vMvP9a060xV,[Scones de la Chola],[1n0013t3w2RbIqSYarnPGS],Una Nueva Realidad,6NCW2haZteRywEWZSzc7in,"{'danceability': 0.536, 'energy': 0.571, 'key'...",[]
628,Dale!,48EI8HkseqMBYQw8yl4WL3,[La Extrema Vanguardia],[3p1OOKD3Rs8JsT9I76mACt],Epe,4Bgue5pbIMGEZ61SoULBMr,"{'danceability': 0.464, 'energy': 0.864, 'key'...",[]
637,Si Me Dijeras,0NhFqADNG4OABBBEtxW0WM,[Vozenoff],[0hASTHk8Lmdj2zAHvkfsfW],Si Me Dijeras,6gyIUgOHK85AQswoDcLDDw,"{'danceability': 0.534, 'energy': 0.747, 'key'...",[]


In [207]:
#Amount of nulls.
songs_original_df[[x == [] for x in songs_original_df["genres"]]].count()

song_name         19
song_id           19
artists           19
artists_id        19
album_name        19
album_id          19
audio_features    19
genres            19
dtype: int64

These cells don't have a genre since Spotify hasn't assigned a genre to their corresponding artists. 
These edge cases will be corrected by hand trying to find the closest match by listening to each song and finding a similiar artist that has an assigned genre in Spotify, if that is possible.

In [209]:
sp.artist("4Apvih9OZt9ghebGFIVcXI")
get_genres(["64DFKvGarD5nmkfaIiiakf"])

['cuarteto']

In [210]:
sanitized_df = songs_original_df.copy()
sanitized_df.iloc[48]["genres"] = get_genres(["7Cg2eqV6oHNE0P54WfajIX"]) #like julio sosa
sanitized_df.iloc[478]["genres"] = 'folklore'
#sanitized_df.iloc[498]["genres"] = get_genres(["4Apvih9OZt9ghebGFIVcXI"]) #like los del suquia
sanitized_df.iloc[526]["genres"] = get_genres(["0iutktJLkNNtErs8c3EoF6"]) #like los tekis
sanitized_df.iloc[600]["genres"] = get_genres(["7okwEbXzyT2VffBmyQBWLz"]) #like maná
sanitized_df.iloc[604]["genres"] = get_genres(["3HrbmsYpKjWH1lzhad7alj"]) #like los autenticos decadentes
sanitized_df.iloc[606]["genres"] = get_genres(["1vumST4jmwQla7lkbLKDUw"]) #like foxley
sanitized_df.iloc[613]["genres"] = get_genres(["5f5Wlazt9jmI75fK5nPpd6"]) #like 8 segundos
sanitized_df.iloc[617]["genres"] = get_genres(["0SnyKkoyBaB2fG8IJH4xmU"]) #like los piojos
sanitized_df.iloc[628]["genres"] = get_genres(["0SnyKkoyBaB2fG8IJH4xmU"]) #like los piojos
sanitized_df.iloc[637]["genres"] = get_genres(["2wjmZuSHtRx96Qxb8HiP2o"]) #like los guasones
sanitized_df.iloc[647]["genres"] = get_genres(["0SnyKkoyBaB2fG8IJH4xmU"]) #like los piojos
sanitized_df.iloc[652]["genres"] = get_genres(["54YdJC33Ztc1CNIuodmyUb"]) #like leo garcia
sanitized_df.iloc[663]["genres"] = get_genres(["1AvkrI2S7knrbaZxydvc9B"]) #like mala fama

sanitized_df.iloc[1504]["genres"] = get_genres(["64DFKvGarD5nmkfaIiiakf"]) #like la mona
sanitized_df.iloc[1521]["genres"] = get_genres(["64DFKvGarD5nmkfaIiiakf"]) #like la mona
sanitized_df.iloc[1523]["genres"] = get_genres(["64DFKvGarD5nmkfaIiiakf"]) #like la mona
sanitized_df.iloc[1526]["genres"] = get_genres(["64DFKvGarD5nmkfaIiiakf"]) #like la mona
sanitized_df.iloc[1538]["genres"] = get_genres(["64DFKvGarD5nmkfaIiiakf"]) #like la mona
sanitized_df.iloc[1541]["genres"] = get_genres(["64DFKvGarD5nmkfaIiiakf"]) #like la mona
sanitized_df[[x == [] for x in sanitized_df["genres"]]]

,song_name,song_id,artists,artists_id,album_name,album_id,audio_features,genres


In [211]:
sanitized_df.count()

song_name         1755
song_id           1755
artists           1755
artists_id        1755
album_name        1755
album_id          1755
audio_features    1755
genres            1755
dtype: int64

Let's check there are no nulls in any other column

In [212]:
columns = sanitized_df.columns

for column in columns:
    if sanitized_df[[x == [] for x in sanitized_df[column]]].empty:
        print("No nulls in column ", column)
    else:
        print(">>>>>>> Found nulls in column", column)

No nulls in column  song_name
No nulls in column  song_id
No nulls in column  artists
No nulls in column  artists_id
No nulls in column  album_name
No nulls in column  album_id
No nulls in column  audio_features
No nulls in column  genres


Let's remove possible duplicated songs

In [213]:
sanitized_df[sanitized_df.duplicated(subset=["song_id"])==True].count()[0]


20

We find duplicated tracks of the same song_id in the dataset.
Let's remove them

In [214]:
songs_df = sanitized_df[sanitized_df["song_id"].duplicated(keep='last') != True]
if not songs_df.duplicated(subset=["song_id"]).any():
    print(f"no duplicated records found in {len(songs_df)} records")

no duplicated records found in 1735 records


Let's make a backup file to not have to run all that code again

In [18]:
#songs_df.to_csv("./songs_df.csv")
#songs_df = pd.read_csv("../songs_df.csv",sep="#")


In [216]:
audio_features_base = pd.DataFrame(list(songs_original_df["audio_features"].values))
audio_features_description = audio_features_base.describe()
audio_features_description

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1.755000e+03,1755.000000
mean,0.585561,0.719249,5.274074,-6.488591,0.611966,0.070666,0.237802,0.031040,0.241244,0.619261,126.232073,2.307568e+05,3.937322
std,0.153703,0.180207,3.544727,2.817251,0.487441,0.064718,0.229648,0.130680,0.226563,0.234545,30.503676,8.823704e+04,0.310502
min,0.078000,0.033500,0.000000,-28.480000,0.000000,0.022100,0.000002,0.000000,0.022000,0.029200,60.026000,3.893300e+04,1.000000
25%,0.491000,0.608000,2.000000,-7.784500,0.000000,0.033200,0.049900,0.000000,0.097500,0.436000,97.884500,1.913570e+05,4.000000
50%,0.608000,0.752000,5.000000,-6.057000,1.000000,0.046600,0.170000,0.000002,0.142000,0.650000,128.048000,2.222830e+05,4.000000
75%,0.696500,0.863000,9.000000,-4.582000,1.000000,0.078600,0.360500,0.000299,0.309000,0.816500,148.078000,2.560925e+05,4.000000
max,0.945000,0.995000,11.000000,-0.767000,1.000000,0.514000,0.982000,0.944000,0.991000,0.976000,205.938000,1.500920e+06,5.000000


## Lyrics

We present the lyrics retrieval and sentiment analysis done at the same time, but first, some considerations.

In [225]:
songs_df.count()

song_name         1735
song_id           1735
artists           1735
artists_id        1735
album_name        1735
album_id          1735
audio_features    1735
genres            1735
dtype: int64

## Considerations, problems found, future work

### Sentiment analysis of spanish texts?

At the time this exploration was done, there weren't many sentiment analysis libraries or tools available for spanish texts. The one used for this jupyter notebook was the only one available, and it required a lot of dependency tinkering to make it work. 

> If you want to run this notebook and you are having problems with that library I recommend you recreate a virtualenv with the same specs as a colab notebook.

This lack of diversity could have impacted in many subtle ways the development of the exploration since there should be done a previous step to check this library's performance against the particular spanish dialects of the dataset. This couldn't be done as we had limited time and it wasn't the main focus of the exploration. It remains an open question how does it perform with bolivian folkloric music and its dialect? with costa rican reggaeton? with argentinian cumbia villera or cuarteto or tango? A lot of investigations could be made based on these questions.

### How to perform sentiment analysis of a song?

A song's lyrics are a long text, with clearly separated phrases or sentences.

How to perform sentiment analysis of a song with this library was explored. 

It was explored whether this library performed better if used on 
- single words
- sentences
- whole lyric's text

in the first two cases, it was also explored how to better come up with the final sentiment analysis score of the song, whether to use the mean or the median.

The following function can be used to make this comparision:

In [57]:
def song_url_for_request(artist, song_title):
    return "https://api.lyrics.ovh/v1/" + artist + '/' + song_title #str
    # example use:
    # requests.get(song_url_for_request("Death Grips", "Hacker"))
    
def compare_performance_mean_vs_median_vs_whole_song(*, artist_name,song_name):
    print(song_name, " by: ", artist_name)
    response = requests.get(song_url_for_request(artist_name,song_name)).json()
    json_data = response 
    lyrics_raw = json_data["lyrics"]
    whole_lyrics_way = lyrics_raw.replace("\n", " ") #for whole sentence
    lyrics_raw = lyrics_raw.split("\n") #for each sentence
    # lo siguiente es por si se necesita analizar palabra por palabra
    #lyrics_raw = [sentence.split(" ") for sentence in lyrics_raw]
        
    sentiments_tmp = []
    for sentence in lyrics_raw:
        thing = sentiment.sentiment(sentence.lower())
        sentiments_tmp.append(thing)
    whole_song_sent = sentiment.sentiment(whole_lyrics_way)
    sent_mean = mean(sentiments_tmp)
    sent_median = median(sentiments_tmp)
    print("mean ",sent_mean, " median ",sent_median)
    print("full song's text: ", whole_song_sent)
    return

In [74]:
songs_df.head(10)

,song_name,song_id,artists,artists_id,album_name,album_id,audio_features,genres
0,Desafío,7j9DYPyCuvSAtPcevpAkzb,['Arca'],['4SQdUpG4f7UbkJG3cJ2Iyj'],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.161, 'energy': 0.482, 'key'...","['art pop', 'dance pop', 'deconstructed club',..."
1,Anoche,1cwTMSQeMaA9fVKEF1iWeD,['Arca'],['4SQdUpG4f7UbkJG3cJ2Iyj'],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.23, 'energy': 0.434, 'key':...","['art pop', 'dance pop', 'deconstructed club',..."
2,Sin Rumbo,0aL27vskbMpwsMGUkHm3Zf,['Arca'],['4SQdUpG4f7UbkJG3cJ2Iyj'],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.289, 'energy': 0.28, 'key':...","['art pop', 'dance pop', 'deconstructed club',..."
3,La Gata Bajo la Lluvia,2kfSFdq2h0xLXq01em1zc7,['Rocío Dúrcal'],['2uyweLa0mvPZH6eRzDddeB'],Sus 16 Grandes Exitos,1QXxmsxolhkqiFtI1mpX4i,"{'danceability': 0.499, 'energy': 0.648, 'key'...","['bolero', 'cancion melodica', 'grupera', 'lat..."
4,Querida,5ySxlyvySBhIEvoO2xx7uT,['Juan Gabriel'],['2MRBDr0crHWE5JwPceFncq'],Recuerdos II,1xrQ48Vvnvm3SmAbnIukGt,"{'danceability': 0.528, 'energy': 0.383, 'key'...","['cancion melodica', 'latin', 'latin pop']"
5,Otro Día para Ser,4c3ix86CIDIVzXBTRUNwBd,['Hermetica'],['6j6Ld5h0aFgH0VQWQNazS7'],Victimas del Vaciamiento,70Bq990gBdpLDzx7M8r28i,"{'danceability': 0.33, 'energy': 0.574, 'key':...","['argentine heavy metal', 'argentine metal', '..."
6,Robo un Auto,6JnnF3ZIpuJt51oG9wVAEb,['Hermetica'],['6j6Ld5h0aFgH0VQWQNazS7'],Acido Argentino,33XPcITW6McI85shkX51RP,"{'danceability': 0.406, 'energy': 0.947, 'key'...","['argentine heavy metal', 'argentine metal', '..."
7,El Pibe Tigre,6p5SbKcAnIECU3hoFkZlPQ,['Almafuerte'],['6qYd7xlmeeeDkPfx6mZ9PV'],Mundo Guanaco,5IyVKrEhTZhhTbUsB8xdl6,"{'danceability': 0.577, 'energy': 0.85, 'key':...","['argentine heavy metal', 'argentine metal', '..."
8,Deseando Destruir y Matar,2zumIZU09Iz2eGRJ5UXHms,['V8'],['2GOgGMMJooNV8Yk2PjvzAa'],Un Paso Mas en la Batalla,0zHdtlc987CwJR3Pr5TV5z,"{'danceability': 0.288, 'energy': 0.982, 'key'...","['argentine heavy metal', 'argentine metal', '..."
9,Nido De Almas,2lkEfig7F9vg1MALElBd0z,['Malon'],['3MBsvBr8B6mfjO6txfT6uL'],Serie De Oro,1KUXCG2YMNtEPZx8MkZ1E9,"{'danceability': 0.467, 'energy': 0.891, 'key'...","['argentine heavy metal', 'argentine metal', '..."


In order to be able to interpret the results I'm going to be comparing two songs with lyrics that I'm very familiar with:
    - Nino Bravo's Un Beso y Una Flor: A song declaring undying love, and flaming optimism.
    - Arca's Sin Rumbo: A very poetic, melancolic song, like the name suggests, about falling out of love but remaining strong.


In [70]:
compare_performance_mean_vs_median_vs_whole_song(artist_name="Nino Bravo",song_name="Un Beso Y Una Flor")

Un Beso Y Una Flor  by:  Nino Bravo
mean  0.48972183  median  0.50473505
full song's text:  0.9976045


In [73]:
compare_performance_mean_vs_median_vs_whole_song(artist_name="Arca",song_name="Sin Rumbo")

Sin Rumbo  by:  Arca
mean  0.50473505  median  0.50473505
full song's text:  0.50473505


The library was able to detect the tone correctly when fed the full lyrics' text. Upon further inspections, it was detected that the library gives outright wrong scores to short sentences, or single words.

Now let's continue building the dataset.

In [228]:
# aux methods
def song_url_for_request(artist, song_title):
    return "https://api.lyrics.ovh/v1/" + artist + '/' + song_title #str
    # example use:
    # requests.get(song_url_for_request("Death Grips", "Hacker"))
    

def add_lyrics_to_data(all_songs):
    songs_without_lyrics = []
    without_lyrics = 0
    for song in tqdm.tqdm(all_songs):
        try:
            response = requests.get(song_url_for_request(song["artists"][0],song["song_name"])).json()
            json_data = response #json.loads(response.content)
            lyrics_raw = json_data["lyrics"]
            lyrics_raw = lyrics_raw.replace("\n", " ") #for whole sentence
            song_sentiment = sentiment.sentiment(lyrics_raw)
            song["lyrics_sentiment"]= round(song_sentiment,4)
        except: #found a song without lyrics!
            without_lyrics +=1
            song["lyrics_sentiment"]=None
            songs_without_lyrics.append(song)
    print(f"There are {without_lyrics} songs without lyrics")
    return all_songs

In [229]:
all_songs = songs_df.copy()
all_songs = all_songs.T.to_dict().values()
all_songs = add_lyrics_to_data(all_songs)


100%|██████████| 1735/1735 [47:45<00:00,  1.65s/it] 

There are 815 songs without lyrics


In [230]:
songs_base = pd.DataFrame(all_songs)
#del songs_base["lyrics"]
#songs_base.to_csv("./songs_base_with_lyrics.csv")

The lyrics API returned a lot of nulls. Let's sanitize.

In [231]:
print(f"Before sanitization {len(all_songs)}")
songs_base.sample(5)

Before sanitization 1735


,song_name,song_id,artists,artists_id,album_name,album_id,audio_features,genres,lyrics_sentiment
1612,Navidad en los Santos (Ao Vivo No el Mapa de T...,5LFhRllOsot55dcBRKt7ql,[El Mató a un Policía Motorizado],[5rLsN2LxYaEPLa1N7I2mPB],El Nuevo Magnetismo (2003 - 2011),21n1dmvxnphIAvf2AlktH9,"{'danceability': 0.388, 'energy': 0.757, 'key'...","[argentine alternative rock, argentine indie, ...",NaN
1473,Te pido que la dejes,0cH2nR2ISmkLVW7e7ogTYw,[Damián Córdoba],[7e3BSeFzlCGyAdbsUbgUaD],Inigualable,1Yq5RNlCkTE1sygpnEstl9,"{'danceability': 0.688, 'energy': 0.811, 'key'...",[cuarteto],0.0000
1436,Mujer Que Mal Me Pagas,3AkjGkz4mT0fvOLMSUlCyV,[Los Palmeras],[2Htm0q72SjGSmsy8EOIhRG],Mujer Que Mal Me Pagas,3OScm1nKAb2EMUKI1n8Bj6,"{'danceability': 0.738, 'energy': 0.794, 'key'...","[argentine rock, cumbia pop, cumbia santafesin...",NaN
1394,Ya Me Voy,0SMjGAyMcvxiTokLeKqyn2,[Walter Olmos],[68mkWeq03e98dvIEI0jr0N],20 Grandes Exitos,1JqbIf2a2CQ3symhTPOKUO,"{'danceability': 0.491, 'energy': 0.855, 'key'...",[cuarteto],NaN
339,Ahora Imagino Cosas,5iQfFljjs2rwXYe68V9VyE,[El Mató a un Policía Motorizado],[5rLsN2LxYaEPLa1N7I2mPB],La Sintesis O'Konor,5YCFp2g2ZZUc0bKNAWyYHN,"{'danceability': 0.494, 'energy': 0.915, 'key'...","[argentine alternative rock, argentine indie, ...",0.0003


In [233]:
# sanitize
songs_sanitiz = songs_base.copy()
bool_series = pd.notnull(songs_sanitiz["lyrics_sentiment"])
songs_base = songs_sanitiz[bool_series]
print(f"After sanitization {len(songs_base)}")
songs_sanitiz.head(3)

After sanitization 920


,song_name,song_id,artists,artists_id,album_name,album_id,audio_features,genres,lyrics_sentiment
0,Desafío,7j9DYPyCuvSAtPcevpAkzb,[Arca],[4SQdUpG4f7UbkJG3cJ2Iyj],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.161, 'energy': 0.482, 'key'...","[art pop, dance pop, deconstructed club, elect...",0.0013
1,Anoche,1cwTMSQeMaA9fVKEF1iWeD,[Arca],[4SQdUpG4f7UbkJG3cJ2Iyj],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.23, 'energy': 0.434, 'key':...","[art pop, dance pop, deconstructed club, elect...",0.9970
2,Sin Rumbo,0aL27vskbMpwsMGUkHm3Zf,[Arca],[4SQdUpG4f7UbkJG3cJ2Iyj],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.289, 'energy': 0.28, 'key':...","[art pop, dance pop, deconstructed club, elect...",0.0000


In [285]:
songs_sanitiz["genres"].sample(10)

1062    [argentine heavy metal, argentine metal, argen...
973           [latin, latin hip hop, reggaeton, tropical]
1200                         [cumbia pop, cumbia villera]
634     [argentine rock, latin alternative, latin rock...
427     [argentine indie, argentine metal, argentine r...
779     [latin, latin arena pop, latin pop, mexican po...
589      [argentine rock, rock en espanol, rock nacional]
1037    [argentine heavy metal, argentine metal, argen...
412     [argentine indie, argentine metal, argentine r...
534     [latin, latin alternative, latin pop, latin ro...
Name: genres, dtype: object

In [234]:
songs_base.to_csv("./final_big.csv",sep="#",index=False)


In [3]:
songs_base = pd.read_csv("./final_big.csv",sep="#")
#let's get back our genres
songs_base["genres"]=songs_base["genres"].apply(lambda x: x.replace("[","").replace("]","").replace("'","").split(","))
songs_base.to_csv("./final_big.csv",sep="#",index=False)


In [4]:
songs_base.head(5)

,song_name,song_id,artists,artists_id,album_name,album_id,audio_features,genres,lyrics_sentiment
0,Desafío,7j9DYPyCuvSAtPcevpAkzb,['Arca'],['4SQdUpG4f7UbkJG3cJ2Iyj'],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.161, 'energy': 0.482, 'key'...","[art pop, dance pop, deconstructed club, el...",0.0013
1,Anoche,1cwTMSQeMaA9fVKEF1iWeD,['Arca'],['4SQdUpG4f7UbkJG3cJ2Iyj'],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.23, 'energy': 0.434, 'key':...","[art pop, dance pop, deconstructed club, el...",0.9970
2,Sin Rumbo,0aL27vskbMpwsMGUkHm3Zf,['Arca'],['4SQdUpG4f7UbkJG3cJ2Iyj'],Arca,1MQO4j8QExVgmnplbIodEU,"{'danceability': 0.289, 'energy': 0.28, 'key':...","[art pop, dance pop, deconstructed club, el...",0.0000
3,La Gata Bajo la Lluvia,2kfSFdq2h0xLXq01em1zc7,['Rocío Dúrcal'],['2uyweLa0mvPZH6eRzDddeB'],Sus 16 Grandes Exitos,1QXxmsxolhkqiFtI1mpX4i,"{'danceability': 0.499, 'energy': 0.648, 'key'...","[bolero, cancion melodica, grupera, latin, ...",0.0057
4,Querida,5ySxlyvySBhIEvoO2xx7uT,['Juan Gabriel'],['2MRBDr0crHWE5JwPceFncq'],Recuerdos II,1xrQ48Vvnvm3SmAbnIukGt,"{'danceability': 0.528, 'energy': 0.383, 'key'...","[cancion melodica, latin, latin pop]",0.0000


# Conclusions

In this notebook we have shown how to construct a dataset of songs, with song features and lyrics sentiment analysis. We are left with a dataset with which we can further explore different topics regarding spanish songs. Problems found were discussed and further work has been proposed.